<a href="https://colab.research.google.com/github/maryantonopoulou/teliki_ergasia_EDDE2/blob/main/newsbomb_teasers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from datetime import datetime

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_teaser_elements_from_articlesL(articlesL):
    articles_dataL = []

    for article in articlesL:
        # Τίτλος
        title_tag = article.find("h3", class_="item-title")
        title = title_tag.text.strip() if title_tag else ""

        # Article URL
        try:
            article_url = title_tag.find("a")["href"] if title_tag else ""
        except:
            article_url = ""

        # Ημερομηνία και Ώρα

        try:
            date_str = article.find("span", {"class": "item-date"}).text.strip()
            time_str = ""
            time_tag = article.find("item-date")
        except:
            date_str = ""
            time_str = ""

        # Excerpt (καμία πραγματική περίληψη στο search teaser, οπότε placeholder ή το subtitle αν υπάρχει)
        excerpt = ""
        excerpt_tag = article.find("div", class_="item-intro")
        if excerpt_tag:
            excerpt = excerpt_tag.text.strip()

        # Section
        section = ""
        category_tag = article.find("span", class_="item-category")
        if category_tag:
            section = category_tag.text.strip()

        # Author (συνήθως δεν υπάρχει στο teaser)
        author = ""

        articleD = {
            "title": title,
            "article_url": article_url,
            "date": date_str,
            "excerpt": excerpt,
            "section": section
        }

        articles_dataL.append(articleD)

    return articles_dataL

In [ ]:
def scrape_newsbomb_teaser_pages(search_term, start_page_nbr, end_page_nbr):

  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1):

    url =f"https://www.newsbomb.gr/search?q={search_term}&page={page_nbr}&q={search_term}&section=oles-oi-eidhseis"

    print(f"\rScraping page {page_nbr}: {url}" + " "*20, end="")

    soup = get_webpage_soup(url)


    articlesL = soup.find("div", {"class": "page-section news-items-section"}).find_all("div", {"class": "card"})

    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    all_articles_dataL.extend(page_articles_dataL)

    delay = random.uniform(1, 3)
    time.sleep(delay)


  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
ergatika_atiximata_df = scrape_newsbomb_teaser_pages("εργατικο%20ατυχημα", 1, 20)

Scraping page 1: https://www.newsbomb.gr/search?q=εργατικο%20ατυχημα&page=1&q=εργατικο%20ατυχημα&section=oles-oi-eidhseis                    https://www.newsbomb.gr/search?q=εργατικο%20ατυχημα&page=1&q=εργατικο%20ατυχημα&section=oles-oi-eidhseis was scraped successfully
Scraping page 2: https://www.newsbomb.gr/search?q=εργατικο%20ατυχημα&page=2&q=εργατικο%20ατυχημα&section=oles-oi-eidhseis                    https://www.newsbomb.gr/search?q=εργατικο%20ατυχημα&page=2&q=εργατικο%20ατυχημα&section=oles-oi-eidhseis was scraped successfully
Scraping page 3: https://www.newsbomb.gr/search?q=εργατικο%20ατυχημα&page=3&q=εργατικο%20ατυχημα&section=oles-oi-eidhseis                    https://www.newsbomb.gr/search?q=εργατικο%20ατυχημα&page=3&q=εργατικο%20ατυχημα&section=oles-oi-eidhseis was scraped successfully
Scraping page 4: https://www.newsbomb.gr/search?q=εργατικο%20ατυχημα&page=4&q=εργατικο%20ατυχημα&section=oles-oi-eidhseis                    https://www.newsbomb.gr/search?q=εργατικο%20ατ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ergatika_atiximata_df.to_csv('/content/drive/MyDrive/CSVFILES/newsbomb_teasers.csv')